## Hybrid Deep Learning Model : CNN & Transformer

> Student : Sasha Annabel (22/496780/PA/21361)

> Topic : Classifying xray medical images to decide whether person has pneumonia or not.

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np


In [ ]:
img_height, img_width = 224, 224  # Resize images to fit model input
batch_size = 32

train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/sem5/deep learning/xray_dataset_covid19/train',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/sem5/deep learning/xray_dataset_covid19/test',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary'
)

Found 148 images belonging to 2 classes.
Found 40 images belonging to 2 classes.


In [ ]:
def build_hybrid_model(input_shape=(img_height, img_width, 3)):
    inputs = layers.Input(shape=input_shape)

    # CNN feature extractor
    x = layers.Conv2D(32, (3, 3), activation='relu')(inputs)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(64, (3, 3), activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(128, (3, 3), activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)

    # Flatten and reshape for transformer input
    x = layers.Flatten()(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Reshape((8, 16))(x)  # Adjust based on the output shape of your CNN

    # Transformer encoder
    transformer_layer = layers.MultiHeadAttention(num_heads=2, key_dim=16)
    x = transformer_layer(x, x)
    x = layers.GlobalAveragePooling1D()(x)

    # Fully connected for classification
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)

    model = models.Model(inputs, outputs)
    return model

model = build_hybrid_model()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator
)


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


5/5 ━━━━━━━━━━━━━━━━━━━━ 85s 9s/step - accuracy: 0.5599 - loss: 0.7255 - val_accuracy: 0.5000 - val_loss: 0.6908
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 28s 4s/step - accuracy: 0.4266 - loss: 0.6974 - val_accuracy: 0.7500 - val_loss: 0.6927
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 29s 4s/step - accuracy: 0.4923 - loss: 0.6942 - val_accuracy: 0.5000 - val_loss: 0.6907
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 28s 4s/step - accuracy: 0.5039 - loss: 0.6934 - val_accuracy: 0.5000 - val_loss: 0.6922
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 39s 4s/step - accuracy: 0.4993 - loss: 0.6918 - val_accuracy: 0.5250 - val_loss: 0.6848
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 41s 4s/step - accuracy: 0.5917 - loss: 0.6806 - val_accuracy: 0.8250 - val_loss: 0.6524
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 27s 4s/step - accuracy: 0.6624 - loss: 0.6654 - val_accuracy: 0.9750 - val_loss: 0.4113
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 41s 4s/step - accuracy: 0.6940 - loss: 0.6023 - val_accuracy: 0.7250 - val_loss: 0.4551
Epoch 9/10
5/5 ━━━━

In [ ]:
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc}")

2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 268ms/step - accuracy: 0.9729 - loss: 0.1145
Test accuracy: 0.9750000238418579
